VappStore
GlobalVappManager
GlobalCommandManager
VappManager
Vapp
VappContextD
CommandPalatte



Command
CommandInputD
CommandOutputD
CommandHOutD
CommandVOutD

CommnadGroup

CommandsParser
CommandExecutor

AICollaborator
AIQuickHelper
HumanCollaborator

Assistant
TaskExecutorLoop

In [ ]:
1. vapp and commands, context configuration files. 
2. vapp init and commands loading.
3. Command Versioning



In [ ]:
c3("page_data") + a1(C3.data) + A1[] + c1("Keys.RETURN") + c2("hgks@gmail.com")

In [ ]:
import abc

class IPrompt(abc.ABC):
    
    @abc.abstractmethod
    def get_value(self):
        pass

class IAICollaborator(abc.ABC):
    
    @abc.abstractmethod
    def send_message(self,prompt : IPrompt):
        pass 
    


In [99]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import csv
from io import StringIO

def chrome_page_data_command(vapp):
    javascript =  """
            function getElementXPath(elt) {
                var path = "";
                for (; elt && elt.nodeType == 1; elt = elt.parentNode) {
                    var idx = getElementIdx(elt);
                    var xname = elt.tagName;
                    if (idx > 1) xname += "[" + idx + "]";
                    path = "/" + xname + path;
                }
                return path;
            }

            function getElementIdx(elt) {
                var count = 1;
                for (var sib = elt.previousSibling; sib ; sib = sib.previousSibling) {
                    if (sib.nodeType == 1 && sib.tagName == elt.tagName) count++;
                }
                return count;
            }

            return getElementXPath(arguments[0]);
            """
    def page_elements(input_dict=None):
        driver = vapp['context']['driver']
        if not driver:
            print("WebDriver instance not found in context store.")
            return None
        
        # Switch to the specified tab
        tab_index = input_dict.get('tab_index', 0)
        try:
            driver.switch_to.window(driver.window_handles[tab_index])
        except IndexError:
            print(f"Error: Tab with index {tab_index} does not exist.")
            return None

        initial_element = driver.find_element(By.TAG_NAME, "body")
        # List to store data related to focused elements
        data = []
        full_data = []
        headers = ['ELEMENT_INDEX', 'TAG', 'TEXT', 'ID', 'NAME', 'INPUT_TYPE', 'VALUE', 'PLACEHOLDER']

        # Define XPath expressions for each type of element
        xpath_expressions = [
            "//input",
            "//a",
            "//select",
            "//option",
            "//button",
            "//form",
            "//textarea",
        ]
        x = 0
        for xpath_expr in xpath_expressions:
            elements = driver.find_elements(By.XPATH, xpath_expr)
            for i, focused_element in enumerate(elements):
                # Extract relevant information
                tag_name = focused_element.tag_name
                class_name = focused_element.get_attribute("class")
                input_value = focused_element.get_attribute("value")
                id = focused_element.get_attribute("id")
                placeholder = focused_element.get_attribute("placeholder")
                name = focused_element.get_attribute("name")
                text = focused_element.text if tag_name != "body" else None
                input_type = focused_element.get_attribute("type") if tag_name == "input" else None
                href = focused_element.get_attribute("href") if tag_name == "a" else None
                xpath = "XPATH: "+driver.execute_script(javascript, focused_element)
                
                obj = {
                    "ELEMENT_INDEX": x,
                    "TAG": tag_name,
                    "TEXT": text,
                    "ID": id,
                    "NAME": name,
                    "VALUE": input_value,
                    "PLACEHOLDER": placeholder,
                    "INPUT_TYPE": input_type,
                }
                
                data.append(obj)
                obj_copy = obj.copy()
                obj_copy['HREF'] = href
                obj_copy['XPATH'] = xpath
                full_data.append(obj_copy)
                x = x + 1

        csv_buffer = StringIO()
        writer = csv.DictWriter(csv_buffer, fieldnames=headers)
        writer.writeheader()
        for item in data:
            writer.writerow(item)

        # Get CSV data as a string
        csv_data = csv_buffer.getvalue()
        
        return {
            "csv_data" : csv_data,
            "full_data": full_data,
            "data": data
        }
    return page_elements

def chrome_click_command(vapp):
    """
    Input parameters:
        - input_dict['xpath']: XPath of the element to click
    Output parameters:
        - None
    """
    def click(input_dict=None):
        driver = vapp['context']['driver']
        xpath = input_dict['xpath']
        element = driver.find_element(By.XPATH, xpath)
        element.click()
        return {}
    return click

def chrome_sleep_command(vapp):
    """
    Input parameters:
        - input_dict['duration']: Duration to sleep in seconds
    Output parameters:
        - None
    """
    def sleep(input_dict=None):
        time.sleep(input_dict['duration'])
        return {}
    return sleep

def chrome_send_keys_command(vapp):
    """
    Input parameters:
        - input_dict['xpath']: XPath of the element to send keys
        - input_dict['keys']: Keys to send
    Output parameters:
        - None
    """
    keys_map = {}

    for key_name in dir(Keys):
        if key_name.isupper():  # Check if the attribute is an uppercase constant
            key_value = getattr(Keys, key_name)
            keys_map['key:'+key_name.lower()] = key_value
    
    def send_keys(input_dict=None):
        xpath = input_dict['xpath']
        keys_to_send = ""
        for key in input_dict['keys']:
            if key.startswith('key:'):  # Check if the argument starts with 'key:'
                key_name = key.split(':')[1]  # Extract the key name
                if key_name.lower() in keys_map:  # Check if the key name is in the keys_map
                    keys_to_send += keys_map[key_name.lower()]  # Append the corresponding Keys constant
            else:
                keys_to_send += key  # If not 'key:', append the argument as it is
        
        driver = vapp['context']['driver']
        element = driver.find_element(By.XPATH, xpath)
        element.send_keys(keys_to_send)
        return {}
    return send_keys

def chrome_open_url_command(vapp):
    """
    Input parameters:
        - input_dict['url']: URL to open
    Output parameters:
        - None
    """
    def open_url(input_dict=None):
        driver = vapp['context']['driver']
        driver.get(input_dict['url'])
        return {}
    return open_url



def chrome_start_command(vapp):
    def start(input_dict=None):
        vapp['context']['driver'] = webdriver.Chrome()
        return {}
    return start

def chrome_quit_command(vapp):
    def quit(input_dict=None):
        vapp['context']['driver'].quit()
        return {}
    return quit

def chrome_init(vapp):
    chrome_vapp['commands']['click'] = chrome_click_command(vapp)
    chrome_vapp['commands']['sleep'] = chrome_sleep_command(vapp)
    chrome_vapp['commands']['start'] = chrome_start_command(vapp)
    chrome_vapp['commands']['open_url'] = chrome_open_url_command(vapp)
    chrome_vapp['commands']['send_keys'] = chrome_send_keys_command(vapp)
    chrome_vapp['commands']['page_data'] = chrome_page_data_command(vapp)
    chrome_vapp['commands']['quit'] = chrome_quit_command(vapp)
    return



chrome_vapp = {
    "context" : {},
    "commands" : {},
    "init_script" : chrome_init
}

chrome_vapp['init_script'](chrome_vapp)

In [96]:
import os

# Commands
def userdata_append_command(vapp):
    """
    Input parameters:
        - input_dict['file_path']: File path
        - input_dict['content']: Content to append
        - input_dict['word_count'] (optional): Word count to update
    Output parameters:
        - status: 'success' or 'error'
        - message: Message indicating the result of the command
    """
    def append(input_dict=None):
        file_path = os.path.join(vapp['context']['root'], input_dict['file_path'])
        content = input_dict['content']
        try:
            with open(file_path, 'a') as file:
                file.write(content + '\n')
            # Update word count
            if input_dict is not None:
                word_count = len(content.split())
                input_dict['word_count'] = word_count
            return {'status': 'success', 'message': 'Appended to file'}
        except Exception as e:
            return {'status': 'error', 'message': str(e)}
    return append


def userdata_create_file_command(vapp):
    """
    Input parameters:
        - input_dict['file_path']: File path
    Output parameters:
        - status: 'success' or 'error'
        - message: Message indicating the result of the command
    """
    def create_file(input_dict=None):
        file_path = os.path.join(vapp['context']['root'], input_dict['file_path'])
        try:
            if not os.path.exists(file_path):
                with open(file_path, 'w'):
                    pass
                return {'status': 'success', 'message': 'File created'}
            else:
                return {'status': 'error', 'message': 'File already exists'}
        except Exception as e:
            return {'status': 'error', 'message': str(e)}
    return create_file

def userdata_get_content_command(vapp):
    """
    Input parameters:
        - input_dict['file_paths']: List of file paths
    Output parameters:
        - status: 'success' or 'error'
        - content: Content of the specified files concatenated together
        - message: Message indicating the result of the command
    """
    def get_content(input_dict=None):
        file_paths = input_dict['file_paths']
        if not file_paths:
            return {'status': 'error', 'message': 'No file paths provided'}
        
        try:
            content = ""
            for file_path in file_paths:
                full_path = os.path.join(vapp['context']['root'], file_path)
                if os.path.exists(full_path):
                    with open(full_path, 'r') as file:
                        content += file.read() + '\n'
                else:
                    return {'status': 'error', 'message': f'File not found: {file_path}'}
            return {'status': 'success', 'content': content}
        except Exception as e:
            return {'status': 'error', 'message': str(e)}
    return get_content


def userdata_list_keys_command(vapp):
    """
    Input parameters: None
    Output parameters:
        - status: 'success' or 'error'
        - keys: List of file names in the user data root folder
        - message: Message indicating the result of the command
    """
    def list_keys(input_dict=None):
        try:
            keys = []
            for root, _, files in os.walk(vapp['context']['root']):
                for file in files:
                    keys.append(file)
            return {'status': 'success', 'keys': keys}
        except Exception as e:
            return {'status': 'error', 'message': str(e)}
    return list_keys

def userdata_init(vapp):
    """
    Initialize the user data vApp.
    Creates a root folder for user data if it doesn't exist, and registers commands.
    """
    # Create a root folder if it doesn't exist
    root_folder = 'user_data'
    if not os.path.exists(root_folder):
        os.makedirs(root_folder)
    vapp['context']['root'] = root_folder
    
    # Register commands
    vapp['commands']['append'] = userdata_append_command(vapp)
    vapp['commands']['create_file'] = userdata_create_file_command(vapp)
    vapp['commands']['get_content'] = userdata_get_content_command(vapp)
    vapp['commands']['list_keys'] = userdata_list_keys_command(vapp)
    return

# Initialization
user_data_vapp = {
    "context": {},
    "commands": {},
    "init_script": userdata_init
}


# AVIL Expression
avil_expression = """
(c1("credentials", "username password") + c1("create_file", "backup_credentials") + c2("credentials") + c1("append", "backup_credentials", "C2_1['content']"))
"""



In [98]:
user_data_vapp['init_script'](user_data_vapp)

# Test create_file command
create_file_input = {'file_path': 'test.txt'}
create_file_result = user_data_vapp['commands']['create_file'](input_dict=create_file_input)
print("Create File Command Result:", create_file_result)

# Test append command
append_input = {'file_path': 'credentials.txt', 'content': 'Hello, world!'}
append_result = user_data_vapp['commands']['append'](input_dict=append_input)
print("Append Command Result:", append_result)

# Test get_content command
get_content_input = {'file_paths': ['test.txt', 'credentials.txt']}
get_content_result = user_data_vapp['commands']['get_content'](input_dict=get_content_input)
print("Get Content Command Result:", get_content_result)

# Test list_keys command
list_keys_result = user_data_vapp['commands']['list_keys']()
print("List Keys Command Result:", list_keys_result)


Create File Command Result: {'status': 'error', 'message': 'File already exists'}
Append Command Result: {'status': 'success', 'message': 'Appended to file'}
Get Content Command Result: {'status': 'success', 'content': '\nNew content for credentials file.Hello, world!\nHello, world!\nHello, world!\n\n'}
List Keys Command Result: {'status': 'success', 'keys': ['credentials.txt', 'work_experience_skills.txt', 'test.txt', 'work_experience.txt']}


In [105]:
import re

class CommandExecutor:
    def __init__(self):
        self.command_history = []
        self.command_outputs = {}

    def execute_expression(self, commands_mapping, expression, vapps, input_dict):
        expression = self.preprocess_expression(expression)
        tokens = self.tokenize_expression(expression)
        parsed_expression = self.parse_expression(tokens)
        return self.evaluate_expression(commands_mapping, parsed_expression, vapps, input_dict)

    def preprocess_expression(self, expression):
        expression = expression.replace('(', ' ( ').replace(')', ' ) ')
        expression = expression.replace('+', ' plus ').replace('-', ' minus ')
        expression = re.sub(r'\s+', ' ', expression)
        return expression.strip()

    def tokenize_expression(self, expression):
        return expression.split()

    def parse_expression(self, tokens):
        stack = []
        for token in tokens:
            if token == '(':
                stack.append([])
            elif token == ')':
                sub_expression = stack.pop()
                if stack:
                    stack[-1].append(sub_expression)
                else:
                    return sub_expression
            else:
                stack[-1].append(token)
        return stack[0]

    def evaluate_expression(self, commands_mapping, parsed_expression, vapps, input_dict):
        if isinstance(parsed_expression, list):
            operator = parsed_expression[0]
            if operator == 'plus':
                return self.evaluate_plus(commands_mapping, parsed_expression[1:], vapps, input_dict)
            elif operator == 'minus':
                return self.evaluate_minus(commands_mapping, parsed_expression[1:], vapps, input_dict)
            elif operator.startswith('?'):
                return self.evaluate_ternary(commands_mapping, parsed_expression[1:], vapps, input_dict)
        else:
            command_name = parsed_expression[0]
            arguments = self.parse_arguments(parsed_expression[1:])
            return self.execute_single_command(commands_mapping, command_name, arguments, vapps, input_dict)

    def evaluate_plus(self, commands_mapping, expressions, vapps, input_dict):
        result = {}
        for expression in expressions:
            result.update(self.evaluate_expression(commands_mapping, expression, vapps, input_dict))
        return result

    def evaluate_minus(self, commands_mapping, expressions, vapps, input_dict):
        result = {}
        for expression in expressions:
            undo_command = '-' + expression[0]
            undo_result = self.execute_command(commands_mapping, undo_command, vapps, input_dict)
            result[undo_command] = undo_result
        return result

    def evaluate_ternary(self, commands_mapping, expressions, vapps, input_dict):
        condition = expressions[0]
        true_expression = expressions[1]
        false_expression = expressions[2]
        condition_result = self.evaluate_expression(commands_mapping, condition, vapps, input_dict)
        if condition_result.get('status') == 'success':
            return self.evaluate_expression(commands_mapping, true_expression, vapps, input_dict)
        else:
            return self.evaluate_expression(commands_mapping, false_expression, vapps, input_dict)

    def parse_arguments(self, arguments):
        return [arg.strip('"') for arg in arguments]

    def execute_single_command(self, commands_mapping, command_name, arguments, vapps, input_dict):
        vapp_name, action_name = commands_mapping[command_name].split(':')
        vapp = vapps.get(vapp_name, {})
        action = getattr(vapp.get('commands', {}), action_name, lambda *args, **kwargs: {})
        output = action(*arguments, input_dict=input_dict)
        self.command_outputs[command_name.upper()] = output
        return output


# Example usage
executor = CommandExecutor()

# Sample AVIL expression
avil_expression = """(c1("credentials", "username password") + c3("backup_credentials") + c1("backup_credentials", "C2_1['content']"))"""



# Sample commands mapping
commands_mapping = {'c1': "userdata:append", 'c2': "userdata:get_content", 'c3' : "userdata:create_file"}

# Sample vapps dictionary
vapps = {
    'chrome': chrome_vapp,
    'userdata': user_data_vapp
}

# Execute the AVIL expression
result = executor.execute_expression(commands_mapping, avil_expression, vapps, {})
print(result)



KeyboardInterrupt: 

In [102]:
chrome_vapp['commands']['start'](input_dict={})
chrome_vapp['commands']['open_url'](input_dict={"url": "https://linkedin.com"})
chrome_vapp['commands']['sleep'](input_dict={"duration": 2})
chrome_vapp['commands']['send_keys'](input_dict={"xpath": "/HTML/BODY/MAIN/SECTION/DIV/DIV/FORM/DIV/DIV/DIV/DIV/INPUT", "keys": "sarath.devarakonda22@gmail.com"})
chrome_vapp['commands']['sleep'](input_dict={"duration": 2})
chrome_vapp['commands']['send_keys'](input_dict={"xpath": "/HTML/BODY/MAIN/SECTION/DIV/DIV/FORM/DIV/DIV[2]/DIV/DIV/INPUT", "keys": "412,./dec"})
chrome_vapp['commands']['sleep'](input_dict={"duration": 2})
chrome_vapp['commands']['send_keys'](input_dict={"xpath": "/HTML/BODY/MAIN/SECTION/DIV/DIV/FORM/DIV/DIV[2]/DIV/DIV/INPUT", "keys": "key:enter"})


{}

In [101]:
chrome_vapp['commands']['quit']()


{}

In [111]:
class Node:
    def __init__(self, value):
        self.value = value
        self.children = []

    def __repr__(self):
        return f"Node({self.value})"


def parse_expression(expression):
    root = Node('root')
    stack = [root]

    # Split the expression string into tokens
    tokens = expression.split()

    for token in tokens:
        if token == '(':
            group = Node('group')
            stack[-1].children.append(group)
            stack.append(group)
        elif token == ')':
            stack.pop()
        elif token == '+':
            if len(stack) < 2 or stack[-2].value == 'group':
                raise ValueError("Invalid expression: '+' operator found at invalid position")
            stack[-2].children.append(Node('+'))
        else:
            # Parse command and its parameters
            command = {}
            parts = token.split('(')
            command['command'] = parts[0]
            if len(parts) > 1:
                params = parts[1].strip(')').split(',')
                for param in params:
                    key_value = param.split('=')
                    if len(key_value) == 2:
                        key, value = key_value
                        command[key.strip()] = value.strip()
                    else:
                        raise ValueError("Invalid parameter format")
            else:
                raise ValueError("Invalid command format")
            # Append command node to the current node in the stack
            stack[-1].children.append(Node(command))

    if len(stack) > 1:
        raise ValueError("Invalid expression: Unmatched parenthesis")

    return root


def print_tree(node, depth=0):
    print('  ' * depth + str(node.value))
    for child in node.children:
        print_tree(child, depth + 1)


# Example usage:
expression_str = "c1(p1=i1) + c2(p1=i1, p2=i2) + c3() + (c1(p1=i3)) + c2(p1=i3, p2=o3.m1)"
tree = parse_expression(expression_str)
print_tree(tree)


ValueError: Invalid expression: '+' operator found at invalid position

In [52]:
page = chrome_vapp['commands']['page_data']()


In [53]:
page['csv_data']

'ELEMENT_INDEX,TAG,TEXT,ID,NAME,INPUT_TYPE,VALUE,PLACEHOLDER\r\n0,input,,jobs-search-box-keyword-id-ember226,,text,,\r\n1,input,,,,text,,"Search by title, skill, or company"\r\n2,input,,,,text,,Search jobs / Location\r\n3,input,,jobs-search-box-location-id-ember226,,text,,\r\n4,input,,,,text,,"City, state, or zip code"\r\n5,a,,,,,,\r\n6,a,Home,,,,,\r\n7,a,"2\n2 new network updates notifications\nMy Network",,,,,\r\n8,a,Jobs,,,,,\r\n9,a,Messaging,,,,,\r\n10,a,"23\n23 new notifications\nNotifications",,,,,\r\n11,a,Try Premium for $0,,,,,\r\n12,a,My jobs,,,,,\r\n13,a,Preferences,,,,,\r\n14,a,Demonstrate skills,,,,,\r\n15,a,Interview prep,,,,,\r\n16,a,Resume Builder,,,,,\r\n17,a,Job seeker guidance,,,,,\r\n18,a,Application settings,,,,,\r\n19,a,Post a free job,ember274,,,,\r\n20,a,,,,,,\r\n21,a,401K match,,,,,\r\n22,a,Parental leave,,,,,\r\n23,a,Food & Bev,,,,,\r\n24,a,Female-founded,,,,,\r\n25,a,,,,,,\r\n26,a,,,,,,\r\n27,a,,,,,,\r\n28,a,,,,,,\r\n29,a,Java Technical Lead,ember288,,,,\r\n30

In [59]:
chrome_vapp['commands']['send_keys'](page['full_data'][0]['XPATH'][7:], "Backend Engineer")
chrome_vapp['commands']['send_keys'](page['full_data'][3]['XPATH'][7:], "New York")



{}

In [47]:
page['csv_data']

'ELEMENT_INDEX,TAG,TEXT,ID,NAME,INPUT_TYPE,VALUE,PLACEHOLDER\r\n0,input,,,,text,,Search\r\n1,input,,msg-overlay-list-bubble-search__search-typeahead-input,,text,,Search messages\r\n2,a,,,,,,\r\n3,a,Home,,,,,\r\n4,a,"2\n2 new network updates notifications\nMy Network",,,,,\r\n5,a,Jobs,,,,,\r\n6,a,Messaging,,,,,\r\n7,a,"23\n23 new notifications\nNotifications",,,,,\r\n8,a,Try Premium for $0,,,,,\r\n9,a,"Welcome, Krrish!",ember25,,,,\r\n10,a,Add a photo,ember27,,,,\r\n11,a,"Connections\nGrow your network",ember29,,,,\r\n12,a,"Strengthen your profile with an AI writing assistant\nTry Premium for $0",,,,,\r\n13,a,My items,ember32,,,,\r\n14,a,"Groups\nSee all Groups",,,,,\r\n15,a,"Events\nSee all Events",,,,,\r\n16,a,"Followed Hashtags\nSee all Followed Hashtags",,,,,\r\n17,a,Discover more,ember40,,,,\r\n18,a,,ember50,,,,\r\n19,a,Event,ember53,,,,\r\n20,a,Write article,ember54,,,,\r\n21,a,,,,,,\r\n22,a,"LinkedIn News\nLinkedIn News\n14,653,170 followers\n14,653,170 followers",,,,,\r\n23,a,"3

In [1]:
def user

def user_data_init(vapp):
    print("hello")
    return

user_data_vapp = {
    "context" : {},
    "commands": {},
    "init_script" : user_data_init
}

In [ ]:
vappstore = {
    "context" : {
        "store_vapps" : {},
        "installed_vapps" : {}
    },
    "commands" : {}
}

vappstore["store_vapps"]["chrome"] = chrome_vapp
vappstore["store_vapps"]["user_data"] = user_data_vapp

vappsmanager_vapp =  {
    "context" : {},
    "commands" : {}
}
    
